# Ερωτηση 3 | 3.Ζ

In [1]:
import pandas as pd
import scipy as sp 
from scipy import stats
import numpy as np
import seaborn as sns
from scipy.spatial.distance import cosine

In [2]:
df_bus = pd.read_csv("business.csv") # Businesses Dataframe
df_res =  pd.read_csv('restaurants.csv') # Restaurants Dataframe
df_red = pd.read_csv('RedfinPhila.tsv', sep='\t')

print("Businesses:", df_bus.shape)  # Businesses have one more column than Restaurants named 'Unnamed: 0.1' 
print("Restaurants:", df_res.shape)
print("Redfin_median_prices:", df_red.shape)



Businesses: (11070, 15)
Restaurants: (4829, 16)
Redfin_median_prices: (45244, 58)


In [3]:
df_bus = df_bus.loc[~df_bus.business_id.isin(df_res.business_id)]
df_bus.dropna(subset=['postal_code'], inplace=True)
df_bus["postal_code"] = df_bus.postal_code.astype(int)
df_bus.postal_code = df_bus.postal_code.astype(str)
df_bus.reset_index(drop=True, inplace=True)

print("Businesses Excluding Restaurants:", df_bus.shape)  # Businesses have one more column than Restaurants named 'Unnamed: 0.1' 
df_bus.head()

Businesses Excluding Restaurants: (6238, 15)


,Unnamed: 0,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,is_open,attributes,categories,hours
0,6,-4dYswJy7SPcbcERvitmIg,Pathmark,3021 Grays Ferry Ave,Philadelphia,PA,19146,39.940403,-75.193297,2.5,34,0,"{'RestaurantsPriceRange2': '2', 'BusinessParki...","Food, Grocery","{'Monday': '6:0-1:0', 'Tuesday': '6:0-1:0', 'W..."
1,8,eMjnw_7wp-CscyNh6Lu0ZA,AM&PM Locksmith,8540 Bustleton Ave,Philadelphia,PA,19152,40.072330,-75.048483,4.5,58,1,"{'ByAppointmentOnly': 'True', 'BusinessAccepts...","Keys & Locksmiths, Home Services, Local Services","{'Monday': '6:0-3:0', 'Tuesday': '6:0-3:0', 'W..."
2,9,xfmkEGiIERBaBMLhw-dTCg,Walmart Vision & Glasses,1675 S Christopher Columbus Blvd,Philadelphia,PA,19148,39.919809,-75.157803,4.5,6,1,"{'RestaurantsPriceRange2': '2', 'BusinessAccep...","Eyewear & Opticians, Shopping","{'Monday': '9:0-20:0', 'Tuesday': '9:0-20:0', ..."
3,11,GWGXTKR0Fhdvzf_isDqJug,"David Gower, Jr. - Coldwell Banker Preferred","325 Chestnut St, Ste 1300",Philadelphia,PA,19106,39.949027,-75.147190,5.0,17,1,"{'BusinessAcceptsCreditCards': 'True', 'ByAppo...","Real Estate, Home Services, Real Estate Agents","{'Monday': '9:0-19:0', 'Tuesday': '9:0-19:0', ..."
4,12,-aeZuatjCDMV1X4gCTz9Ug,David Thomas Trailways,14005 McNulty Rd,Philadelphia,PA,19154,40.106409,-74.973937,4.5,6,1,"{'WiFi': ""u'free'""}","Buses, Transportation, Bus Tours, Hotels & Tra...","{'Monday': '8:30-17:0', 'Tuesday': '8:30-17:0'..."


In [4]:
df_cat_freq = pd.DataFrame(df_bus.groupby(by="postal_code")["categories"].apply(lambda c: c.str.split(',', expand=True).stack().value_counts()))
print(df_cat_freq.shape)
df_cat_freq.head()

(10727, 1)


categories
postal_code                            
19003       Contractors               1
             Local Services           1
             Home Services            1
             Painters                 1
             Handyman                 1

In [5]:
initial_categories = set(df_bus["categories"].str.split(',', expand=True).stack().value_counts().index)
different_categories = set()
for category  in initial_categories:
    category = category.strip()
    different_categories.add(category)

# print(different_categories.shape)
print(len(different_categories), len(initial_categories))
# What happened to the first?

824 1341


In [6]:
def set_categories(attr):
    try:
        attr = attr.categories
        categories = "" 
        for i in attr:
            categories += i
        categories = categories.split(",")
        all_categories = dict.fromkeys(different_categories, 0)
        for category in categories:
            category = category.strip()
            try:        # Normally this try shouldnt exist, since all_categories should contain every key category.
                all_categories[category] += 1       
            except:
                pass
        return all_categories
    except:
        return None

In [7]:
df_pc_cat = pd.DataFrame(df_bus.groupby(by="postal_code").apply(set_categories))
print(df_pc_cat.shape)
df_pc_cat.rename(columns = {0:'num_of_categories'}, inplace = True)
df_pc_cat["postal_code"] = df_pc_cat.index
df_pc_cat.reset_index(drop=True, inplace=True)
df_pc_cat.head()


(80, 1)


,num_of_categories,postal_code
0,"{'Real Estate Services': 0, 'Used Car Dealers'...",19003
1,"{'Real Estate Services': 0, 'Used Car Dealers'...",19004
2,"{'Real Estate Services': 0, 'Used Car Dealers'...",19006
3,"{'Real Estate Services': 0, 'Used Car Dealers'...",19010
4,"{'Real Estate Services': 0, 'Used Car Dealers'...",19012


Για να δουμε ποιες περιοχες εχουν κοινη οικονομικη κατασταση θα δουμε το Redfin Median Sale Prices 

In [8]:
df_red = df_red[["median_sale_price", "region"]]
df_red["region"] = df_red.region.apply(lambda s: s.strip("Zip Code: "))
df_red.rename(columns = {'region':'postal_code'}, inplace = True)
df_red = df_red.groupby(by="postal_code")["median_sale_price"].mean()
# df_red.index = df_red.postal_code
df_red = pd.DataFrame(df_red)
print(df_red.shape)
df_red.head()

(106, 1)


,median_sale_price
postal_code,
19003,348269.932594
19008,309258.622892
19010,469248.786871
19013,52576.980159
19014,207546.861213


In [9]:
df_cat_price = df_pc_cat.merge(df_red, on="postal_code")
df_cat_price.index = df_cat_price.postal_code
df_cat_price = df_cat_price[["num_of_categories", "median_sale_price"]]
print(df_cat_price.shape)
df_cat_price.head()

(52, 2)


,num_of_categories,median_sale_price
postal_code,,
19003,"{'Real Estate Services': 0, 'Used Car Dealers'...",348269.932594
19010,"{'Real Estate Services': 0, 'Used Car Dealers'...",469248.786871
19014,"{'Real Estate Services': 0, 'Used Car Dealers'...",207546.861213
19023,"{'Real Estate Services': 0, 'Used Car Dealers'...",80747.046260
19102,"{'Real Estate Services': 4, 'Used Car Dealers'...",596292.791304


In [10]:
df_by_sale_price = df_cat_price.sort_values("median_sale_price")
df_by_sale_price_reverse = df_cat_price.sort_values("median_sale_price", ascending=False)

df_by_sale_price.shape

(52, 2)

In [11]:
cosine_distances_by_2 = []
cosine_distances_by_2_rev = []

for i in range(49):
    cosine_distances_by_2.append(1 - cosine(pd.Series(df_by_sale_price.num_of_categories[i]), pd.Series(df_by_sale_price.num_of_categories[i+1])))
for i in range(49):
    cosine_distances_by_2_rev.append(1 - cosine(pd.Series(df_by_sale_price_reverse.num_of_categories[i]), pd.Series(df_by_sale_price_reverse.num_of_categories[i+1])))

In [12]:
(np.array(cosine_distances_by_2).mean() + np.array(cosine_distances_by_2_rev).mean()) / 2

0.5334435326657871

## Τελικά Συμπεράσματα
Από εδώ βλέπουμε ότι οι περιοχές που έχουν (γειτονικές) αξίες που δηλαδή είναι κοντινές, έχουν κατά μέσο ορό 0.53 συσχέτιση που είναι μέτριος υψηλή. Άρα, πιστεύουμε ότι υπάρχει κάποια συσχέτιση. Ο τρόπος που έβγαλα τα συμπεράσματα δεν είναι κάλος αν όχι λάθος και δεν υπάρχει στατιστικός έλεγχος, για να βεβαιωθούμε ότι αυτό το αποτέλεσμα δεν μπορεί να προκύψει από ανεξάρτητες τυχαίες μεταβλητές.